<a href="https://colab.research.google.com/github/matthewpblock/applied-ml-matthew-block/blob/main/notebooks/project04/ml04_block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# P4 - Continuous Target Prediction Using Regression (Titanic)
Matthew Block  
15 Nov 2025  
Opening with short intro describing dataset and objectives.

In [ ]:
# Imports
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Section 1: Import and Inspect the Data

## Section 2: Data Exploration and Preparation

## Section 3: Feature Selection and Justification

## Section 4: Train a Regression Model (Linear Regression)

## Section 5: Compare Alternative Models (Ridge, Elastic Net, Polynomial Regression)

## Section 6: Final Thoughts & Insights